In [1]:
import json
from collections import Counter

In [2]:
base_synthetic_responses_path = 'C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Synthetic-Responses-JSON'
base_data_path = 'C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Environmental-Views-Variables'

In [3]:
question_1_json_file = "\\synthetic_responses_demo.json"
question_1_data_file = "\\scenv_crlf\\Environmental Friendly Behaviour Probability Distribution Wave 10.json"

In [4]:
question_2_json_file = "\\q2_synthetic_responses.json"
question_2_data_file = "\\scenv_bccc\\Scenv Bccc Probability Distribution Wave 10.json"

In [5]:
question_3_json_file = "\\q3_synthetic_responses.json"
question_3_data_file = "\\scenv_pmep\\Scenv Pmep Probability Distribution Wave 10.json"

In [6]:
question_4_json_file = "\\q4_synthetic_responses.json"
question_4_data_file = "\\OpenVB\\Open VB Probability Distribution Wave 18.json"

In [7]:
question_5_json_file = "\\q5_synthetic_responses.json"
question_5_data_file = "\\scenv_meds\\Scenv Meds Probability Distribution Wave 10.json"

In [8]:
question_6_json_file = "\\q6_synthetic_responses.json"
question_6_data_file = "\\etariff\\etariff Probability Distribution Wave 10.json"

In [9]:
question_7_json_file = "\\q7_synthetic_responses.json"
question_7_data_file = "\\grimyn\\grimyn Probability Distribution Wave 3.json"

In [10]:
question_8_json_file = "\\q8_synthetic_responses.json"
question_8_data_file = "\\orga3\\orga3 Probability Distribution Wave 3.json"

In [11]:
question_9_json_file = "\\q9_synthetic_responses.json"
question_9_data_file = "\\scenv_tlat\\Scenv Tlat Probability Distribution Wave 10.json"

In [12]:
questions_json_files = [question_1_json_file, question_2_json_file, question_3_json_file, question_4_json_file, question_5_json_file, question_6_json_file, question_7_json_file, question_8_json_file, question_9_json_file]
questions_data_files = [question_1_data_file, question_2_data_file, question_3_data_file, question_4_data_file, question_5_data_file, question_6_data_file, question_7_data_file, question_8_data_file, question_9_data_file]

## MAF1 and Absolute Difference Calculation

In [13]:
def calculate_maf1_and_absolute_difference(base_data_path, question_2_data_file, base_synthetic_responses_path, question_2_json_file):
    # Load data
    with open(base_data_path + question_2_data_file) as f:
        data = json.load(f)

    with open(base_synthetic_responses_path + question_2_json_file) as f:
        responses = json.load(f)

    # Aggregate synthetic responses
    synthetic_counts = Counter()
    for response in responses:
        synthetic_counts.update(response["Synthetic Responses"])

    # Convert counts to proportions
    total_responses = sum(synthetic_counts.values())
    synthetic_proportions = {k: v / total_responses for k, v in synthetic_counts.items()}

    # Ensure all keys in UKHLS data are present in synthetic data
    for key in data.keys():
        if key not in synthetic_proportions:
            synthetic_proportions[key] = 0.0

    # Compute the absolute differences
    absolute_differences = {k: abs(synthetic_proportions[k] - data[k]) for k in data.keys()}

    # Calculate MAF1
    maf1 = sum(absolute_differences.values()) / len(absolute_differences)

    # Print results
    print(maf1)
    print(absolute_differences)

    return maf1, absolute_differences

In [14]:
question_index = 0

sum_maf1 = 0
sum_absolute_differences = Counter()

for question_json_file, data_json_file in zip(questions_json_files, questions_data_files):
    maf1_score, absolute_differences = calculate_maf1_and_absolute_difference(base_data_path, data_json_file, base_synthetic_responses_path, question_json_file)
    print(f"Question {question_index + 1} MAF1 Score: {maf1_score}")
    print(f"Question {question_index + 1} Absolute Differences: {absolute_differences}")

    sum_maf1 += maf1_score
    sum_absolute_differences.update(absolute_differences)

    question_index += 1

print(f"Average MAF1 Score: {sum_maf1 / len(questions_json_files)}")
print(f"Total Absolute Differences: {sum_absolute_differences}")

# Calculate the total sum of absolute differences and the number of differences
total_sum_absolute_differences = 0
total_number_of_differences = 0

for diff, count in sum_absolute_differences.items():
    # Ensure that the difference is a number
    try:
        total_sum_absolute_differences += count
        total_number_of_differences += 1
    except (ValueError, TypeError) as e:
        pass

# Calculate the average of absolute differences
if total_number_of_differences > 0:
    average_absolute_difference = total_sum_absolute_differences / total_number_of_differences
else:
    average_absolute_difference = 0

print(f"Average Absolute Difference: {average_absolute_difference}")

0.20000000000000004
{"Don't do Anything Environmentally Friendly": 0.05744607582255893, 'Do One or Two Things Environmentally Friendly': 0.3565726450239489, 'Do Some Things Environmentally Friendly': 0.40453307453902265, 'Do Many Things Environmentally Friendly': 0.1623203831825439, 'Do Everything Environmentally Friendly': 0.019127821431925617}
Question 1 MAF1 Score: 0.20000000000000004
Question 1 Absolute Differences: {"Don't do Anything Environmentally Friendly": 0.05744607582255893, 'Do One or Two Things Environmentally Friendly': 0.3565726450239489, 'Do Some Things Environmentally Friendly': 0.40453307453902265, 'Do Many Things Environmentally Friendly': 0.1623203831825439, 'Do Everything Environmentally Friendly': 0.019127821431925617}
0.18632
{'Strongly agree': 0.0813, 'Tend to agree': 0.3675, 'Neither': 0.3391, 'Tend to disagree': 0.1186, 'Strongly disagree': 0.0251}
Question 2 MAF1 Score: 0.18632
Question 2 Absolute Differences: {'Strongly agree': 0.0813, 'Tend to agree': 0.36

In [ ]:
q2_maf1, q2_absolute_differences = calculate_maf1_and_absolute_difference(base_data_path, question_2_data_file, base_synthetic_responses_path, question_2_json_file)

print("Question 2 MAF1: ", q2_maf1)
print("Question 2 Absolute Differences: ", q2_absolute_differences)

In [ ]:
# load data 
with open(base_data_path + question_2_data_file) as f:
    data = json.load(f)

with open(base_synthetic_responses_path + question_2_json_file) as f:
    responses = json.load(f)

In [ ]:
# Aggregate synthetic responses
synthetic_counts = Counter()
for response in responses:
    synthetic_counts.update(response["Synthetic Responses"])

In [ ]:
# Convert counts to proportions
total_responses = sum(synthetic_counts.values())
synthetic_proportions = {k: v / total_responses for k, v in synthetic_counts.items()}

In [ ]:
# Ensure all keys in UKHLS data are present in synthetic data
for key in data.keys():
    if key not in synthetic_proportions:
        synthetic_proportions[key] = 0.0

In [ ]:
# Compute the absolute differences
absolute_differences = {k: abs(synthetic_proportions[k] - data[k]) for k in data.keys()}

# Calculate MAF1
maf1 = sum(absolute_differences.values()) / len(absolute_differences)

In [ ]:
print(maf1)

In [ ]:
print(absolute_differences)